## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

3.8.12 (default, Oct 13 2021, 06:42:42) 
[Clang 13.0.0 (clang-1300.0.29.3)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_1Cond/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 20.

FixedMinVal        = 1.e-14

In [3]:
FileName = OutputDir+'/Orig/train/ext/CleanVars.csv'
Vars     = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()[0,:]

In [4]:

Vars0 = []
for Var in Vars:
    Vars0.append(str(Var)+'0')
    

FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
NICs         = len(ICVecs)


P0Vec        = []
EqRatio0Vec  = []
T0Vec        = []
for iC in range(NICs):
    Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
    P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
    EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
    T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(NICs):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)
for iC in range(NICs):
    FileName         = OutputDir+'/Orig/train/ext/y.csv.'+str(iC+1)
    DataNew          = pd.read_csv(FileName, header=0)
    Nts              = len(DataNew)
    DataTemp         = DataNew.iloc[[0]][Vars]#DataTemp.iloc[1][Vars]
    DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
    DataTemp.reset_index(drop=True, inplace=True)
    DataNew[Vars0]   = DataTemp
    DataTemp['t']    = DataNew['t']
    DataNew['t']     = DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy()
    if (iC==0):
        Data        = DataNew
    else:
        Data        = pd.concat([Data, DataNew], axis=0)
Data.reset_index(drop=True, inplace=True)

In [5]:
Data

,t,T,H2,H,O,O2,OH,H2O,HO2,H2O2,...,H2O20,N0,NH0,NH30,NNH0,NO0,NO20,N2O0,HNO0,N20
0,0.000000e+00,1000.000070,0.013802,0.000000e+00,0.000000e+00,0.219059,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
1,3.691671e-11,1000.000070,0.013802,2.915271e-17,1.959178e-17,0.219059,1.260498e-17,1.850911e-17,1.089446e-15,7.629506e-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
2,7.396970e-11,1000.000070,0.013802,5.841304e-17,3.925588e-17,0.219059,2.525649e-17,3.708654e-17,2.182914e-15,1.528718e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
3,1.111595e-10,1000.000070,0.013802,8.778140e-17,5.899258e-17,0.219059,3.795471e-17,5.573256e-17,3.280419e-15,2.297312e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
4,1.484865e-10,1000.000070,0.013802,1.172582e-16,7.880214e-17,0.219059,5.069980e-17,7.444741e-17,4.381976e-15,3.068743e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,9.853686e-01,2160.630241,0.000041,3.970680e-06,3.712669e-04,0.104103,2.905370e-03,1.213814e-01,4.423874e-06,2.061547e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767140
49996,9.890062e-01,2160.621599,0.000041,3.970393e-06,3.712413e-04,0.104100,2.905249e-03,1.213815e-01,4.423663e-06,2.061473e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767140
49997,9.926573e-01,2160.613022,0.000041,3.970108e-06,3.712158e-04,0.104098,2.905128e-03,1.213816e-01,4.423454e-06,2.061399e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767140
49998,9.963219e-01,2160.604510,0.000041,3.969825e-06,3.711906e-04,0.104095,2.905008e-03,1.213816e-01,4.423246e-06,2.061326e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.767140


In [6]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/Orig/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/')
except:
    pass

In [7]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[['t'] + list(Vars0)]
DataInput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + list(Vars)]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)